In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [9]:
#ally_direct = pd.read_csv("./alliances/alliance_v4.1_by_directed.csv")
ally_direct_year = pd.read_csv("./alliances/alliance_v4.1_by_directed_yearly.csv")
#ally_dyad = pd.read_csv("/alliances/alliance_v4.1_by_dyad.csv")
#ally_dyad_year = pd.read_csv("/alliances/alliance_v4.1_by_dyad_yearly.csv")
#ally_member = pd.read_csv("/alliances/alliance_v4.1_by_member.csv")
#ally_member_year = pd.read_csv("/alliances/alliance_v4.1_by_member_yearly.csv")

In [10]:
ally_direct_year.info()
# Double the amount of entries as they are repeated in both directions

ally_direct_year.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148258 entries, 0 to 148257
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   version4id      148258 non-null  int64  
 1   ccode1          148258 non-null  int64  
 2   state_name1     148258 non-null  object 
 3   ccode2          148258 non-null  int64  
 4   state_name2     148258 non-null  object 
 5   dyad_st_day     148258 non-null  int64  
 6   dyad_st_month   148258 non-null  int64  
 7   dyad_st_year    148258 non-null  int64  
 8   dyad_end_day    32708 non-null   float64
 9   dyad_end_month  32708 non-null   float64
 10  dyad_end_year   148190 non-null  float64
 11  left_censor     148258 non-null  int64  
 12  right_censor    148258 non-null  int64  
 13  defense         148258 non-null  int64  
 14  neutrality      148258 non-null  int64  
 15  nonaggression   148237 non-null  float64
 16  entente         148234 non-null  float64
 17  year      

,version4id,ccode1,state_name1,ccode2,state_name2,dyad_st_day,dyad_st_month,dyad_st_year,dyad_end_day,dyad_end_month,dyad_end_year,left_censor,right_censor,defense,neutrality,nonaggression,entente,year,version
0,1,200,United Kingdom,235,Portugal,1,1,1816,NaN,NaN,2012.0,1,1,1,0,1.0,0.0,1816,4.1
1,1,200,United Kingdom,235,Portugal,1,1,1816,NaN,NaN,2012.0,1,1,1,0,1.0,0.0,1817,4.1
2,1,200,United Kingdom,235,Portugal,1,1,1816,NaN,NaN,2012.0,1,1,1,0,1.0,0.0,1818,4.1
3,1,200,United Kingdom,235,Portugal,1,1,1816,NaN,NaN,2012.0,1,1,1,0,1.0,0.0,1819,4.1
4,1,200,United Kingdom,235,Portugal,1,1,1816,NaN,NaN,2012.0,1,1,1,0,1.0,0.0,1820,4.1


In [11]:
ally_direct_year.tail()

,version4id,ccode1,state_name1,ccode2,state_name2,dyad_st_day,dyad_st_month,dyad_st_year,dyad_end_day,dyad_end_month,dyad_end_year,left_censor,right_censor,defense,neutrality,nonaggression,entente,year,version
148253,414,666,Israel,2,United States of America,30,11,1981,26.0,12.0,1991.0,0,0,0,0,0.0,1.0,1987,4.1
148254,414,666,Israel,2,United States of America,30,11,1981,26.0,12.0,1991.0,0,0,0,0,0.0,1.0,1988,4.1
148255,414,666,Israel,2,United States of America,30,11,1981,26.0,12.0,1991.0,0,0,0,0,0.0,1.0,1989,4.1
148256,414,666,Israel,2,United States of America,30,11,1981,26.0,12.0,1991.0,0,0,0,0,0.0,1.0,1990,4.1
148257,414,666,Israel,2,United States of America,30,11,1981,26.0,12.0,1991.0,0,0,0,0,0.0,1.0,1991,4.1


In [12]:
ally_direct_year.isnull().sum()

version4id             0
ccode1                 0
state_name1            0
ccode2                 0
state_name2            0
dyad_st_day            0
dyad_st_month          0
dyad_st_year           0
dyad_end_day      115550
dyad_end_month    115550
dyad_end_year         68
left_censor            0
right_censor           0
defense                0
neutrality             0
nonaggression         21
entente               24
year                   0
version                0
dtype: int64

In [24]:
# Finding asymmetrical alliances
ally_dyad = pd.read_csv("./alliances/alliance_v4.1_by_dyad.csv")
asymmetrical = ally_dyad[ally_dyad['asymmetric'].astype(bool)]

# Search my directed data for these asymmetric alliance pairs in a particular year

selected_year = 2012 # Change this for different years
select_year_df = ally_direct_year[ally_direct_year['year'] == selected_year]

pairs = []
for index, row in asymmetrical.iterrows():
    search_values = [row['ccode1'], row['ccode2'], row['dyad_st_year']]
    mask = (select_year_df[['ccode1','ccode2','dyad_st_year']] == search_values).all(axis=1)
    mask_reverse = (select_year_df[['ccode2','ccode1','dyad_st_year']] == search_values).all(axis=1)

    # Searched for dataframe pair
    results_df = pd.concat([select_year_df[mask], select_year_df[mask_reverse]], ignore_index=True)

    # Remove the empty pairs
    if len(results_df) != 0:
        pairs.append(results_df)


# All the pairs of asymmetrical alliances
#pairs[0].head()
print("There are ",len(pairs)," asymmetric alliances\n")

asymmetric_countries= {}

for pair in pairs:
    # Counter for country 1
    asymmetric_countries[pair['state_name1'].values[0]] = asymmetric_countries.get(pair['state_name1'].values[0], 0) + 1

    # Counter for country 2
    asymmetric_countries[pair['state_name2'].values[0]] = asymmetric_countries.get(pair['state_name2'].values[0], 0) + 1

# Sorting and displaying in descending order countries : values
countries_by_asymmetric = sorted(asymmetric_countries, key = asymmetric_countries.get, reverse = True)

values_by_asymmetric = sorted(asymmetric_countries.values(), reverse = True)
for i in range(len(countries_by_asymmetric)):
    print(countries_by_asymmetric[i],' : ',values_by_asymmetric[i])
#max_key = max(asymmetric_countries, key=asymmetric_countries.get)

# Find the maximum value
#max_value = asymmetric_countries[max_key]

# Display the result
#print("Key with the maximum value:", max_key)
#print("Maximum value:", max_value)

There are  169  asymmetric alliances

Trinidad and Tobago  :  29
Ecuador  :  29
Peru  :  29
Paraguay  :  29
Uruguay  :  29
Turkey  :  19
United States of America  :  10
Canada  :  6
Bahamas  :  5
Haiti  :  5
Dominican Republic  :  5
Jamaica  :  5
Barbados  :  5
Dominica  :  5
Grenada  :  5
St. Lucia  :  5
St. Vincent and the Grenadines  :  5
Antigua & Barbuda  :  5
St. Kitts and Nevis  :  5
Mexico  :  5
Belize  :  5
Guatemala  :  5
Honduras  :  5
El Salvador  :  5
Nicaragua  :  5
Costa Rica  :  5
Panama  :  5
Colombia  :  5
Venezuela  :  5
Guyana  :  5
Suriname  :  5
Brazil  :  5
Bolivia  :  5
Chile  :  5
Argentina  :  5
Denmark  :  2
Iceland  :  2
United Kingdom  :  1
Netherlands  :  1
Belgium  :  1
Luxembourg  :  1
France  :  1
Spain  :  1
Portugal  :  1
Germany  :  1
Poland  :  1
Hungary  :  1
Czech Republic  :  1
Italy  :  1
Greece  :  1
Norway  :  1
Japan  :  1
South Korea  :  1
Pakistan  :  1
Russia  :  1
Armenia  :  1
